## Data ETL 

### Initial data - dictionaries

I asked my friends to provide data in form of a dictionary to ensure correct data structure. Not all of them are coders, but all of them encountered data in life before. I explained that the dictionary is a data structure that for every category (key) has a corresponding value. I asked for a dictionary to look like this:

In [ ]:
my_name = {
    'nickname': '', # Your in-game summoner name
    'tagline':'', # Riot_id tagline
    'role': '', # The role you are gonna play
    'champ_pool': ['champ1', 'champ2', 'champ3', ...] # List you favorite champs but please no more than 6
}

They were not so sure about it and some asked if they can just send a table in some popular program (that should remain nameless). But with my mission of disenchanting coding for people I proposed that I will fill the dictionary based on my data and that they could simply repeform and then they agreed. I knew they could do it!

#### My example

In [ ]:
dariusz_dict = {
    'nickname': 'Vile Vilainess',
    'tagline':'JNX',
    'role': 'jungler',
    'champ_pool': ['Lilia', 'Volibear', 'Sejuani', 'Poppy', 'Ekko', 'Hecarim'] # I know I am already frustrated with myself that i set limit to 6
}

#### My friends Data

In [ ]:
matt_dict = {
    'nickname': 'Cut Desire',
    'tagline':'EUNE',
    'role':'adc',
    'champ_pool': ['Samira', 'Jinx', 'Draven', 'Sivir']
}

mateo_dict = {
    'nickname': 'Mateo Chirimbolo',
    'tagline':'S1E1',
    'role':'top',
    'champ_pool': ['Mordekaiser', 'Olaf', 'Ornn', 'Maokai', 'Gwen', 'Volibear']
}

gustavo_dict = {
    'nickname': 'SztywneSkarpety',
    'tagline':'65881',
    'role':'support',
    'champ_pool': ['Lulu', 'Soraka', 'Nautilius', 'Pyke', 'Poppy', 'Maokai']
}

mitch_dict = {
    'nickname': 'I SAY GG 5 TIMES',
    'tagline':'2137',
    'role':'mid',
    'champ_pool': ['Seraphine', 'Syndra', 'Jayce', 'Sylas', 'Orianna']
}

### List of dictionaries

I thanked my bros for finalizing that ferocious task and created a list of all dictionaries for later use

In [ ]:
players = [dariusz_dict, matt_dict, mateo_dict, gustavo_dict, mitch_dict]

In [8]:
import pandas as pd

# List of player dictionaries
players = [dariusz_dict, matt_dict, mateo_dict, gustavo_dict, mitch_dict]

# Data for the first table
table1_data = {
    'nickname': [],
    'tagline': [],
    'role': []
}

# Data for the second table
table2_data = {
    'role': [],
    'champion': []
}

# Populate the data for both tables
for player in players:
    # Add data to the first table
    table1_data['nickname'].append(player['nickname'])
    table1_data['tagline'].append(player['tagline'])
    table1_data['role'].append(player['role'])
    
    # Add data to the second table
    for champ in player['champ_pool']:
        table2_data['role'].append(player['role'])
        table2_data['champion'].append(champ)

# Create DataFrames
players = pd.DataFrame(table1_data)
champion_pool = pd.DataFrame(table2_data)


## Creating and population an sqlite database

### Creation

In [6]:
import sqlite3 as sql

# Create SQLite database
main_db = sql.connect(r'104 Project DB.db')

# Create an explicit cursor object since SQLite embeeded in Python does not support implicit cursor object
cursor = main_db.cursor()

### Inserting prepared data into a database
We create new table for each prepared dataframe 

In [ ]:
# Insert data from DataFrame into new table in DB
players.to_sql('players', main_db, if_exists='replace', index = True)
champion_pool.to_sql('champion_pool', main_db, if_exists='replace', index = True)

27

## Getting champions ID

Our team obviously operates with champion names but what I know is that Riot API operates with champion IDs. HAving that in mind i got [JSON file](102%20Champions%20data.JSON) from Riot's site for developers that will allow me to connect (relate) champion names to theirs ID. 

#

In [10]:
import pandas as pd
import json

file_path = '102 Champions data.JSON'

with open (file_path, 'r') as file:
    data = json.load(file)


# Extract relevant data for DataFrame
champion_list = []
for champion, details in data['data'].items():
    champ_dict = {
        'id': details['id'],
        'name': details['name'],
        'title': details['title'],
        'attack': details['info']['attack'],
        'defense': details['info']['defense'],
        'magic': details['info']['magic'],
        'difficulty': details['info']['difficulty'],
        'tags': ", ".join(details['tags']),
        'partype': details['partype'],
        'hp': details['stats']['hp'],
        'hpperlevel': details['stats']['hpperlevel'],
        'mp': details['stats']['mp'],
        'mpperlevel': details['stats']['mpperlevel'],
        'movespeed': details['stats']['movespeed'],
        'armor': details['stats']['armor'],
        'armorperlevel': details['stats']['armorperlevel'],
        'spellblock': details['stats']['spellblock'],
        'spellblockperlevel': details['stats']['spellblockperlevel'],
        'attackrange': details['stats']['attackrange'],
        'hpregen': details['stats']['hpregen'],
        'hpregenperlevel': details['stats']['hpregenperlevel'],
        'mpregen': details['stats']['mpregen'],
        'mpregenperlevel': details['stats']['mpregenperlevel'],
        'crit': details['stats']['crit'],
        'critperlevel': details['stats']['critperlevel'],
        'attackdamage': details['stats']['attackdamage'],
        'attackdamageperlevel': details['stats']['attackdamageperlevel'],
        'attackspeedperlevel': details['stats']['attackspeedperlevel'],
        'attackspeed': details['stats']['attackspeed']
    }
    champion_list.append(champ_dict)

# Create DataFrame
champions_df = pd.DataFrame(champion_list)

Now simply add another table into our db

In [11]:
champions_df.to_sql('champions', main_db, if_exists='replace', index=True)

169

### Issue with creating sql tables using Pandas

No primary key